# Characterizing WTSN channels

This notebooks simulates a basic WTSN system with some UEs, an access point and 
some base schedules to determine what the packet latencies and throughput are under 
different conditions.

In [48]:
# Necessary imports

from enum import Enum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import typing
from typing import List, Tuple, Dict

In [68]:
# Create the necessary classes and structures

# TODO: do you need a mapping between the parameters of the sheet and corresponding latency?
# TODO: Add functions to plot the base schedules and whatnot

class PacketStatus(Enum):
    '''
    Enum to represent the status of a packet
    '''
    ARRIVED = 1
    QUEUED = 2
    DELIVERED = 3
    DROPPED = 4

class Packet:
    '''
    Class to represent a packet in the Wi-Fi network
    '''

    def __init__(self, size: int, priority: int, sequence_number: int, arrival_time: float) -> None:
        '''
        Constructor for the packet class

        Args:
            size (int): Size of the packet in bytes
            priority (int): Priority of the packet, the higher the value, higher
                the priority
            arrival_time (float): Time at which the packet arrives at the UE's 
                link layer for transmission in microseconds
        ''' 
        self.size = size
        self.priority = priority
        self.sequence_number = sequence_number
        self.arrival_time = arrival_time
        self.delivery_time = None
        self.status = PacketStatus.ARRIVED

    # Implement a __str__ function to print the packet
    def __str__(self) -> str:
        '''
        Function to print the packet
        '''
        output = (f"Packet(Size: {self.size} bytes, "
            f"Priority: {self.priority}, "
            f"Sequence Number: {self.sequence_number}, "
            f"Arrival Time: {self.arrival_time} microseconds), "
            f"Delivery Time: {self.delivery_time} microseconds), "
            f"Status: {self.status.name}")

        return output
        



class Slot:
    '''
    Class to represent a slot in the Wi-Fi network. This corresponds to a Qbv window
    and can either be reserved for a UE or have contetion
    '''
    
    def __init__(self, slot_index: int, start_time: float, end_time: float, mode: str, 
                UEs: List[str]) -> None:
        '''
        Constructor for the Slot class

        Args:
            slot_index (int): Index of the slot in the schedule, starts from 0
            start_time (float): Start time of the slot in microseconds
            end_time (float): End time of the slot in microseconds
            mode (str): Mode of the slot, can be either contention or reserved (using Qbv)
            UEs (List[str]): List of UEs that are permitted to transmit in the slot
        '''
        self.slot_index = slot_index
        self.start_time = start_time
        self.end_time = end_time
        self.mode = mode
        self.UEs = UEs

    # Implement a __str__ function to print the slot
    def __str__(self) -> str:
        '''
        Function to print the slot
        '''
        return (f"Slot(Index: {self.slot_index}, "
            f"Start Time: {self.start_time} microseconds, "
            f"End Time: {self.end_time} microseconds, "
            f"Mode: {self.mode}, "
            f"UEs: {self.UEs})")

class Schedule:
    '''
    Class to represent the base schedule for the Wi-Fi network
    '''

    def __init__(self, start_time: float, end_time: float, num_slots: int, 
                 schedule: typing.Dict[int, Slot]) -> None:
        '''
        Constructor for the BaseSchedule class

        Args:
            start_time (float): Start time of the schedule in microseconds
            end_time (float): End time of the schedule in microseconds
            num_slots (int): Number of slots in the schedule
            schedule (typing.Dict[int, typing.Dict[int, int]]): A dictionary with 
                the priority as the key and another dictionary as the value. This
                inner dictionary has the sequence number as the key and the size
                of the packet as the value
        '''
        self.start_time = start_time
        self.end_time = end_time
        self.num_slots = num_slots
        self.schedule = schedule

    # Implement a __str__ function to print the schedule   
    def __str__(self) -> str:
        '''
        Function to print the schedule
        '''
        output = (f"Schedule(Start Time: {self.start_time} microseconds, "
            f"End Time: {self.end_time} microseconds, "
            f"Number of Slots: {self.num_slots})")
        output += "\nSlots: \n"
        for slot in self.schedule:
            output += "\t" + str(self.schedule[slot]) + "\n"
        return output

class UE:
    '''
    Class to represent a User Equipment (UE) or STA in the Wi-Fi network
    '''
    # TODO: evaluate use of MCS
    # TODO: make MCS a function of not just priority but also time or slot index
    # TODO: write setting functions for each parameter
    # TODO: consider making this a super class with AP and STA as subclasses
    # TODO: consider adding both uplink and downlink functionality- is it even different
    def __init__(self, ue_id: int, mcs: typing.Dict[int, int], network_mode_of_operation: str, 
                service_mode_of_operation: str, n_packets : int) -> None:
        '''
        Constructor for the UE class

        Args:
            mcs (typing.Dict[int, int]): Dictionary with the priority as the key
                and the MCS as the value i.e packets with the priority will 
                be transmitted at the corresponding MCS
            netowork_mode_of_operation (str): Mode of operation of the UE, can be either
                "central control" or "free" (TODO: add more modes)
            service_mode_of_operation (str): Mode of operation of the UE within a Qbv
                window, used to determine how TXOP is used, how packets are allocated etc.
            n_packets (int): Number of packets that the UE has to transmit
        '''
        self.ue_id = ue_id
        self.mcs = mcs
        self.network_mode_of_operation = network_mode_of_operation
        self.service_mode_of_operation = service_mode_of_operation
        self.n_packets = n_packets
        # List to store the packets that the UE has to transmit
        self.packets = []

    def __str__(self) -> str:
        '''
        Function to print the UE
        '''
        output = (f"UE(UE ID: {self.ue_id}, "
            f"UE(MCS: {self.mcs}, "
            f"Network mode of Operation: {self.network_mode_of_operation}, "
            f"Service mode of Operation: {self.service_mode_of_operation}, "
            f"Number of Packets: {self.n_packets})")
        output += "\nPackets: \n"
        for packet in self.packets:
            output += "\t" + str(packet) + "\n"
        return output
        


    def  generate_packets(self, base_schedule : Schedule, packet_size: List[int]) -> None:
        '''
        TODO: Fix this to take a custom generator function from outside and run that,
            this function can potentially enforce some constraints on the custom funciton that 
            is passed in 
        Function to generate the packets that the UE has to transmit

        Args:
            base_schedule (Schedule): A base schedule specifying
                Qbv windows for different UEs across time
            packet_size (List[int]): size of each packet
        '''

        if self.network_mode_of_operation == "central control":
            # Generate packets based on the schedule
            num_packets_per_slot = [int(np.floor(self.n_packets/base_schedule.num_slots))]*base_schedule.num_slots
            # If the number of packets isn't divisible by the number of slots
            # then put the remaining packets in the last slot
            num_packets_per_slot[-1] += self.n_packets - sum(num_packets_per_slot)

            packet_counter = 0
            for slot_index, num_packets in enumerate(num_packets_per_slot):
                for _ in range(num_packets):
                    self.packets.append(Packet(size=packet_size[packet_counter], 
                                                priority=slot_index+1, 
                                                sequence_number=packet_counter,
                                                arrival_time=base_schedule.schedule[slot_index].end_time - 1))
                    packet_counter += 1

    def obtain_packet_latency(self) -> List[float]:
        '''
        Function to obtain the latency of each packet

        Returns:
            List[float]: List of latencies of each packet
        '''
        latencies = []
        for packet in self.packets:
            latencies.append(packet.delivery_time - packet.arrival_time)
        return latencies
            
    def serve_packets(self, base_schedule: Schedule) -> None:
        # TODO: Move this function to the network class
        '''
        Function to serve the packets that the UE has to transmit

        Args:
            base_schedule (Schedule): A base schedule specifying
                Qbv windows for different UEs across time
        '''
        if self.service_mode_of_operation == "Mode 1":
            '''
            Mode 1: Dummy mode for testing 

            Simply marks all packets as served after a 3000 microsecond delay
            '''

            for packet in self.packets:
                packet.delivery_time = packet.arrival_time + 3000
                packet.status = PacketStatus.DELIVERED




# TODO: Write a network class

In [69]:
# Create a schedule with 2 slots and 2 UEs
slot1 = Slot(0, 0, 10000, "reserved", ["UE1"])
slot2 = Slot(1, 10000, 20000, "reserved", ["UE2"])
schedule = {0: slot1, 1: slot2}
base_schedule = Schedule(0, 20000, 2, schedule)
print(base_schedule)

# Create a UE
ue = UE(0,{1: 0, 2: 1}, "central control", "Mode 1",  10)
ue.generate_packets(base_schedule, [100]*10)
print(ue)

# Serve the packets
ue.serve_packets(base_schedule)
print(ue)

latencies = ue.obtain_packet_latency()
print(latencies)

Schedule(Start Time: 0 microseconds, End Time: 20000 microseconds, Number of Slots: 2)
Slots: 
	Slot(Index: 0, Start Time: 0 microseconds, End Time: 10000 microseconds, Mode: reserved, UEs: ['UE1'])
	Slot(Index: 1, Start Time: 10000 microseconds, End Time: 20000 microseconds, Mode: reserved, UEs: ['UE2'])

UE(UE ID: 0, UE(MCS: {1: 0, 2: 1}, Network mode of Operation: central control, Service mode of Operation: Mode 1, Number of Packets: 10)
Packets: 
	Packet(Size: 100 bytes, Priority: 1, Sequence Number: 0, Arrival Time: 9999 microseconds), Delivery Time: None microseconds), Status: ARRIVED
	Packet(Size: 100 bytes, Priority: 1, Sequence Number: 1, Arrival Time: 9999 microseconds), Delivery Time: None microseconds), Status: ARRIVED
	Packet(Size: 100 bytes, Priority: 1, Sequence Number: 2, Arrival Time: 9999 microseconds), Delivery Time: None microseconds), Status: ARRIVED
	Packet(Size: 100 bytes, Priority: 1, Sequence Number: 3, Arrival Time: 9999 microseconds), Delivery Time: None micr